# 学習率を回転角で正規化

In [32]:
! rm -rd MOD2NN
! git clone https://$$TOKEN@github.com/konnitiha3/MOD2NN.gitA

Cloning into 'MOD2NN'...
remote: Enumerating objects: 1538, done.
remote: Counting objects: 100% (303/303), done.
remote: Compressing objects: 100% (169/169), done.
remote: Total 1538 (delta 166), reused 217 (delta 104), pack-reused 1235:   4% (62/1538), 3.66 MiB | 2.35 MiB/sReceiving objects:   6% (93/1538), 4.89 MiB | 2.37 MiB/sReceiving objects:   8% (124/1538), 4.89 MiB | 2.37 MiB/sReceiving objects:  10% (154/1538), 6.45 MiB | 2.52 MiB/sReceiving objects:  11% (183/1538), 6.45 MiB | 2.52 MiB/sReceiving objects:  13% (200/1538), 7.82 MiB | 2.55 MiB/sReceiving objects:  17% (262/1538), 7.82 MiB | 2.55 MiB/sReceiving objects:  19% (293/1538), 7.82 MiB | 2.55 MiB/sReceiving objects:  21% (323/1538), 8.90 MiB | 2.48 MiB/sReceiving objects:  23% (357/1538), 8.90 MiB | 2.48 MiB/sReceiving objects:  25% (385/1538), 9.52 MiB | 2.32 MiB/sReceiving objects:  27% (416/1538), 9.52 MiB | 2.32 MiB/sReceiving objects:  29% (447/1538), 10.96 MiB | 2.35 MiB/sReceiving objects:  32% (493/1538), 13.2

In [1]:
import sys
sys.path.append('/notebooks/MOD2NN')

In [2]:
import time
import sys
import os
import json

import pandas as pd
from Faraday.two_dim.module.lib.layers import *

tf.random.set_seed(1)

print("TensorFlow:", tf.__version__)
print("Python:", sys.version)

plt.rcParams['font.size'] = 18

TensorFlow: 2.9.1
Python: 3.9.13 (main, May 23 2022, 22:01:06) 
[GCC 9.4.0]


In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

y_train = y_train.astype("float32")
y_test = y_test.astype("float32")

# Reserve 10,000 samples for validation
x_val = x_train[-5000:]
y_val = y_train[-5000:]
x_train = x_train[:-5000]
y_train = y_train[:-5000]

11490434/11490434 [==============================] - 0s 0us/step


In [4]:
#@title デフォルトのタイトル テキスト
wavelength = 633.0e-9 #@param {type:"number"}
d = 1.0e-6 #@param {type:"number"}
n = 1.5 #@param {type:"number"}
z = 0.7e-3 #@param {type:"number"}

In [7]:
def create_model(**kwargs):
    shape = (100, 100)
    theta = kwargs["theta"]
    eta = 0.
    tf.random.set_seed(kwargs["seed"])
    inputs = tf.keras.Input((28, 28))
    x = ImageResizing(shape)(inputs)
    x = IntensityToElectricField(shape)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = MO(shape, limitation='sin', theta=theta, eta=eta)(x)
    x = AngularSpectrum(shape, wavelength=wavelength, z=z, d=d, n=n, method='expand', normalization="max")(x)
    x = FaradayRotationByStokes(shape)(x)
    x = MNISTDetector(shape)(x)
    x = tf.keras.layers.Softmax()(x)
    model = tf.keras.Model(inputs, x)
    return model

In [8]:
def train(seed, theta):
    model_name = "20221130/lr_theta_norm/theta_pi{:d}_seed_{:d}".format(int(np.round(np.pi/theta)), seed)
    path = "/notebooks/"
    checkpoint_path = "checkpoint/" + model_name + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    path = path + "trained_model/"+ model_name

    lr = 0.001 / theta
    model = create_model(seed=seed, theta=theta)
    model.summary()
    model.compile(optimizer=tf.keras.optimizers.Adam(lr),
                  loss=tf.keras.losses.sparse_categorical_crossentropy,  # category: sparse_categorical_crossentropy
                  metrics=["accuracy"]
                  )

    epochs = 30

    # チェックポイントコールバックを作る
    cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)

    logdir = os.path.join(path +"logs", model_name)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    result = model.fit(x_train,
                       y_train,
                       batch_size=64,
                       epochs=epochs,
                       validation_data=(x_val, y_val),
                       callbacks=[cp_callback, tensorboard_callback]
                       )

    model.save(path)

    df = pd.DataFrame(result.history)
    df.to_csv(path + "/history.csv")



    with open(path + "/config.json", 'w') as f:
        json.dump(model.get_config(), f, indent=4)

In [9]:
list = [np.pi/100., np.pi/500., np.pi/1000.]
for theta in list:
    for i in range(5):
        train(seed=i+1, theta=theta)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing (ImageResizi  (None, 100, 100)         0         
 ng)                                                             
                                                                 
 intensity_to_electric_field  (None, 2, 100, 100)      0         
  (IntensityToElectricField)                                     
                                                                 
 angular_spectrum (AngularSp  (None, 2, 100, 100)      0         
 ectrum)                                                         
                                                                 
 mo (MO)                     (None, 2, 100, 100)       10000     
                                                             

INFO:tensorflow:Assets written to: /notebooks/trained_model/20221130/lr_theta_norm/theta_pi100_seed_1/assets


INFO:tensorflow:Assets written to: /notebooks/trained_model/20221130/lr_theta_norm/theta_pi100_seed_1/assets


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 image_resizing_1 (ImageResi  (None, 100, 100)         0         
 zing)                                                           
                                                                 
 intensity_to_electric_field  (None, 2, 100, 100)      0         
 _1 (IntensityToElectricFiel                                     
 d)                                                              
                                                                 
 angular_spectrum_6 (Angular  (None, 2, 100, 100)      0         
 Spectrum)                                                       
                                                                 
 mo_5 (MO)                   (None, 2, 100, 100)       1000

INFO:tensorflow:Assets written to: /notebooks/trained_model/20221130/lr_theta_norm/theta_pi100_seed_2/assets


...

INFO:tensorflow:Assets written to: /notebooks/trained_model/20221130/lr_theta_norm/theta_pi1000_seed_4/assets


INFO:tensorflow:Assets written to: /notebooks/trained_model/20221130/lr_theta_norm/theta_pi1000_seed_4/assets


Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 28, 28)]          0         
                                                                 
 image_resizing_14 (ImageRes  (None, 100, 100)         0         
 izing)                                                          
                                                                 
 intensity_to_electric_field  (None, 2, 100, 100)      0         
 _14 (IntensityToElectricFie                                     
 ld)                                                             
                                                                 
 angular_spectrum_84 (Angula  (None, 2, 100, 100)      0         
 rSpectrum)                                                      
                                                                 
 mo_70 (MO)                  (None, 2, 100, 100)       100

INFO:tensorflow:Assets written to: /notebooks/trained_model/20221130/lr_theta_norm/theta_pi1000_seed_5/assets


INFO:tensorflow:Assets written to: /notebooks/trained_model/20221130/lr_theta_norm/theta_pi1000_seed_5/assets
